# QNA Tool

This is a general tool to help with technical questions with GPT and Llama models.  

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment

load_dotenv(override=True)

True

In [4]:
# prompts

system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please answer the following question:"

In [15]:
# question

question = """
Please explain what this Python code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [16]:
# messages

messages=[
        {
            "role": "user",
            "content": user_prompt + question
        },
        {
            "role": "system",
            "content": system_prompt
        }
    ]

In [7]:
# Get gpt-4o-mini to answer, with streaming

client = OpenAI()

completion = client.chat.completions.create(
    model=MODEL_GPT,
    messages=messages,
    stream=True
)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in completion:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

The given Python code snippet is using the `yield from` statement combined with a set comprehension. Here's a breakdown of what it does:

### Breakdown of the Code

1. **Set Comprehension**: 
   python
   {book.get("author") for book in books if book.get("author")}
   
   - This part creates a set of authors from a collection called `books`.
   - It iterates through each `book` in the `books` iterable.
   - The expression `book.get("author")` retrieves the value associated with the key `"author"` for each `book`. 
   - The `if book.get("author")` condition filters out any `book` entries where the `"author"` key does not exist or has a value that evaluates to `False` (e.g., `None`, an empty string, etc.).
   - As a result, this creates a set of authors that have valid values.

2. **`yield from`**:
   python
   yield from { ... }
   
   - The `yield from` statement is used within a generator function to yield all items from an iterable (in this case, the set of authors).
   - It effectively allows the generator to produce values one at a time from the underlying iterable without needing a loop to do so.

### Purpose of the Code
The overall purpose of this code is to define a generator that yields unique author names from a list of book dictionaries. It provides a mechanism to retrieve authors without duplicates and allows the consuming code to iterate over the authors one by one.

### Example Usage
Assuming `books` is a list of dictionaries structured like this:
python
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three"},  # No author
    {"title": "Book Four", "author": "Author A"},  # Duplicate author
]


After executing the generator function containing the code, the yielded authors would be:
- "Author A"
- "Author B"

### Summary
In summary, this code effectively extracts and yields unique authors from a list of books in a clean and memory-efficient manner using a generator.

In [20]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt + question}
]

# messages

messages=[
            {
            "role": "system",
            "content": system_prompt
            },
    
            {
            "role": "user",
            "content": user_prompt + question
            }
        ]

In [ ]:
# Get Llama 3.2 to answer

from ollama import chat

response = chat(model=MODEL_LLAMA, messages=messages)
display(Markdown(response['message']['content']))